In [1]:
#@title Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import itertools
import pickle
import numpy as np
from numpy import random
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
from tqdm import tqdm
from PIL import Image
from matplotlib import pyplot as plt
import matplotlib
from typing import TypeVar, Dict

import torch
import torch.nn as nn
from torch.optim.optimizer import Optimizer

from counterfactuals.utils import make_dir, get_transforms, torch_to_image, expl_to_image
from counterfactuals.plot import plot_grid_part
from counterfactuals.generative_models.base import GenerativeModel
from counterfactuals.classifiers.base import NeuralNet

Tensor = TypeVar('torch.tensor')

#matplotlib.use('Agg')

import click
from argparse import Namespace
import ast
import os

import torchvision.transforms as transforms
from torch.autograd import Variable
from counterfactuals.classifiers.base import NeuralNet
import torch.nn.functional as F
from typing import TypeVar, Tuple
import counterfactuals.classifiers.cnn as classifiers
import counterfactuals.classifiers.unet as unet
from counterfactuals.utils import load_checkpoint
from counterfactuals.data import get_data_info
from counterfactuals.generative_models.factory import get_generative_model
import gdown
import copy

no display found. Using non-interactive Agg backend


In [3]:
#@title Define Hyperparameters

# class_names = ["5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs",
#                 "Big_Lips", "Big_Nose", "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows",
#                 "Chubby", "Double_Chin", "Eyeglasses", "Goatee", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones",
#                 "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", "Oval_Face", "Pale_Skin",
#                 "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair",
#                 "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace",
#                 "Wearing_Necktie", "Young"]
# num_classes = 40
# data_shape = [3, 64, 64]
# n_bits = 5
# temp = 0.7
# data_mean = [0.485, 0.456, 0.406]
# data_std = [0.229, 0.224, 0.225]

input_size = 28*28*1 # img_size = (28,28) ---> 28*28=784 in total
batch_size = 1000 # the size of input data took for one iteration

In [4]:
# from torchvision.transforms import v2
# transform_RandomErasing=transforms.Compose([v2.RandomErasing(),
#                               transforms.ToTensor()])
#from torchvision.transforms import v2
#transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=0,std=1.0)])
transform=transforms.Compose([transforms.ToTensor()])
train_data = dsets.FashionMNIST(root = './data', train=True, transform = transform, download = True)
test_data = dsets.FashionMNIST(root = './data', train=False, transform = transform, download = True)

In [5]:
#@title Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)



In [6]:
#cvmx=torch.zeros([3*64*64,3*64*64],device=device)
images_list=[]
labels_list=[]
no_batches=len(train_gen)
#images_mean=torch.zeros(3,64,64,device=device)
for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images).cuda().detach()
    labels=Variable(labels).cuda().detach()
    # images_mean=images_mean+images.mean(0)
    # im=torch.reshape(images,[images.shape[0],3*64*64])
    # cvmx+=torch.matmul(torch.transpose(im,0,1),im)
    if(i<(len(train_gen))):
        images_list.append(images)
        labels_list.append(labels)



test_images_list=[]
test_labels_list=[]
test_no_batches=len(test_gen)
for i ,(images,labels) in enumerate(test_gen):
    images = Variable(images).cuda().detach()
    labels=Variable(labels).cuda().detach()
    if(i<(len(test_gen))):
        test_images_list.append(images)
        test_labels_list.append(labels)

train_data_len=len(train_data)
test_data_len=len(test_data)



In [7]:
#@title Define model class
import torch.nn as nn
import torch
from counterfactuals.classifiers.base import NeuralNet
import torch.nn.functional as F
from counterfactuals.utils import load_checkpoint
from counterfactuals.utils import save_checkpoint

from typing import TypeVar, Tuple

Tensor = TypeVar('torch.tensor')


class multinomial(NeuralNet):
    """
    CNN for (binary) classification for CelebA, CheXpert
    """

    def __init__(self,
                 num_classes: int = 10,
                 flattened_size: int = 28*28):
        """Builder."""
        super(multinomial, self).__init__()

        self.last_layer=nn.Sequential(
            nn.Flatten(),
            nn.Linear(flattened_size, num_classes)
        )

    def forward(self, x: Tensor) -> Tensor:
        """Perform forward."""

        x=self.last_layer(x)

        return x

    def classify(self, x: Tensor) -> Tuple[Tensor, Tensor, Tensor]:
        net_out = self.forward(x)
        acc = F.softmax(net_out, dim=1)
        class_idx = torch.max(net_out, 1)[1]

        return acc, acc[0, class_idx], class_idx
    




In [8]:
# net=Fashion_MNIST_CNN()
# n=0
# for par in net.parameters():
#     n+=par.numel()

# n

In [9]:
#@title Define loss-function & optimizer
loss_function = nn.CrossEntropyLoss()


def images_regulariser(net): 
    li_reg_loss = 0
    penalized     = [p for name,p in net.named_parameters() if 'bias' not in name]
    not_penalized = [p for name,p in net.named_parameters() if 'bias' in name]
    for p in penalized:
        li_reg_loss += (p**2).sum()*0.5
    #for p in net.parameters():
#        li_reg_loss += (p**2).sum()*0.5
    reg=li_reg_loss/(train_data_len)*l2regconst
    return(reg)


In [10]:
def addnet(net,net2):
    for param1, param2 in zip(net.parameters(), net2.parameters()):
     param1.data += param2.data

def multiplynet(net,a):
   for param1 in net.parameters():
     param1.data *=a

In [11]:

def U(x,v,hc,xi1,xi2):

    xn=x+hc.etam1g*v+hc.c11*xi1
    vn=v*hc.eta+hc.c21*xi1+hc.c22*xi2
    return([xn, vn])

def bounce(x,v,xstar,width):
    vsign=(((x-xstar+width)/(2*width)).floor()% 2)*(-2)+1
    vn=v*vsign
    xn=((x-xstar-width)% (4*width)-2*width).abs()-width+xstar  
    return([xn, vn])

def bouncenet():
    for p,p_star in zip(net.parameters(),net_star.parameters()):
        [p.data, p.v]=bounce(p.data, p.v, p_star.data, 6/torch.sqrt(l2regconst_extra))

def svrg_grad(net, batch_it):
    outputsU = net(images_list[batch_it])
    loss_likelihood = loss_function(outputsU, labels_list[batch_it])  


    grads_reg=[torch.zeros_like(par) for par in net.parameters()]
    net_pars=list(net.parameters())
    with torch.no_grad():
        for it in range(len_params):
            if(list_no_bias[it]):
                grads_reg[it]=net_pars[it].data*l2regconst

    net.zero_grad()
    loss_likelihood.backward()
    with torch.no_grad():
        grads_likelihood=[par.grad*batch_size for par in net.parameters()]

        svrg_grads=[]
        for p,grad,grad_reg,p_star,grad_star,star_sum_grad in zip(list(net.parameters()),grads_likelihood,grads_reg,list(net_star.parameters()),net_star_grad_list[batch_it],net_star_full_grad):              
            svrg_grads.append(grad_reg+star_sum_grad+(grad-grad_star)*no_batches+l2regconst_extra*(p.data-p_star.data))
    return svrg_grads,loss_likelihood.data

def full_grad(net):
    outputs = net(all_images)
    net.zero_grad()
    full_loss = (loss_function(outputs, all_labels)+images_regulariser(net))*train_data_len
    full_loss.backward()
    grads=[par.grad for par in net.parameters()]

    return grads,full_loss.data


def leapfrogvxv_step(net,h,batch_it):   

    with torch.no_grad():
        for p in net.parameters():
            p+=(h/2)*p.v

    svrg_grads,_=svrg_grad(net, batch_it)

    with torch.no_grad():
        for p,grad in zip(net.parameters(), svrg_grads):
            p.v-=h*grad
            p+=(h/2)*p.v

def leaponlyv(net,h,batch_it):   
    svrg_grads,_=svrg_grad(net, batch_it)

    with torch.no_grad():
        for p,grad in zip(net.parameters(), svrg_grads):
            p.v-=h*grad


def leapforward(net,h,batch_it):   
    svrg_grads,_=svrg_grad(net, batch_it)

    with torch.no_grad():
        for p,grad in zip(net.parameters(), svrg_grads):
            p.v-=h*grad
    with torch.no_grad():
        for p in net.parameters():
            p.data+=(h*no_batches/(no_batches-1))*p.v

def leapbackward(net,h,batch_it):   
           #p.data+=(h/2)*p.v
    with torch.no_grad():
        for p in net.parameters():
            p.data+=(h*no_batches*2/(no_batches*2-1))*p.v
    svrg_grads,_=svrg_grad(net, batch_it)
    with torch.no_grad():
        for p,grad in zip(net.parameters(), svrg_grads):
            p.v-=h*grad

def leapfrog_step(net,h):   
    grads,_=full_grad(net)
    
    with torch.no_grad():
        for p,grad in zip(net.parameters(), grads):
            p.v-=(h/2)*grad

    with torch.no_grad():
        for p in net.parameters():
            p.data+=(h)*p.v

    grads,_=full_grad(net)

    with torch.no_grad():
        for p,grad in zip(net.parameters(), grads):
            p.v-=(h/2)*grad


def hamiltonian(net):
    with torch.no_grad():
        outputs = net(all_images)
        ham=loss_function(outputs, all_labels)*train_data_len+images_regulariser(net)*train_data_len
        for par in net.parameters():
            ham+=par.v.pow(2).sum()/2
        return(ham)



def ind_create(batch_it):
    modit=batch_it %(2*no_batches)
    ind=(modit<=(no_batches-1))*modit+(modit>=no_batches)*(2*no_batches-modit-1)
    return ind


In [12]:
#net = Fashion_MNIST_CNN().cuda()
#net2=copy.deepcopy(net)
#addnet(net2,net)
#multiplynet(net2,1/10)

In [13]:
# filepath="output_fashion_low_rank_9n.pickle"
# #filepath="output_fashion_low_rank_long.pickle"
# with open(filepath,"rb") as file:
#    [labels_arr,test_labels_arr,test_prob_arr]=pickle.load(file)
# labels_arr=torch.tensor(labels_arr).detach()
# test_labels_arr=torch.tensor(test_labels_arr).detach()
# test_prob_arr=torch.tensor(test_prob_arr).detach()

In [14]:
# Outcome should be a binary list of the ordinal outcome. [0, 1, 0] for exmaple.
# Probs should be a list of probabilities. [0.79, 0.09, 0.12] for example.
# Outcome and Probs must be provided with the same order as probabilities.

def rps_single(probs, true_label):
    outcome=torch.zeros(num_classes)
    outcome[true_label.int()]=1.0
    cum_probs = torch.cumsum(probs,0)
    cum_outcomes = torch.cumsum(outcome,0)
    
    #print(cum_outcomes)
    #print(cum_probs)
    sum_rps = 0
    for i in range(len(outcome)):         
        sum_rps+= (cum_probs[i] - cum_outcomes[i])**2
    
    return sum_rps/(num_classes-1)

def rps_calc(test_probs, true_labels):
    rps_vec=torch.zeros(test_data_len)
    for it in range(test_data_len):
        rps_vec[it]=rps_single(test_probs[it,:].reshape(num_classes),true_labels[it])
    return rps_vec

def nll_calc(test_probs, true_labels):
    res=0
    for it in range(test_data_len):
        res-=torch.log(test_probs[it,true_labels[it].int()])
    return res/test_data_len

def adaptive_calibration_error(test_probs,true_labels, num_bins=20):
    max_probs, predicted_labels = torch.max(test_probs,1)
    ind=torch.argsort(max_probs,stable=True)
    sorted_max_probs=max_probs[ind]
    sorted_predicted_labels=predicted_labels[ind]
    sorted_true_labels=true_labels[ind]

    correct = (sorted_predicted_labels == sorted_true_labels).clone().detach().float()
    bins=(torch.tensor(range(test_data_len))/torch.tensor(test_data_len/num_bins)).floor()

    o=torch.tensor(0.0)
    for b in range(num_bins):
        mask = (bins == b)
        if torch.any(mask):
            #print(b, sorted_max_probs[mask].mean(), (correct[mask] - sorted_max_probs[mask]).mean())
            o += (correct[mask] - sorted_max_probs[mask]).mean().abs()

    return o / num_bins

In [15]:
#@title Output arrays
num_classes=10

training_size=no_batches*batch_size
test_size=test_data_len


#l2regconst=torch.tensor(50).detach()
l2regconst=torch.tensor(10).detach()
l2regconst_extra=torch.tensor(0).detach()
gam=torch.sqrt(l2regconst)
#hper2c=hper2const(torch.tensor(h/2),gam)
all_images=torch.cat(images_list,dim=0).detach()
all_labels=torch.cat(labels_list,dim=0).detach()
# net = multinomial().cuda()
# net2 = multinomial().cuda()
# net.train()
# net2.train()

def find_MAP(num_steps):
  net = multinomial().cuda()

  def lpost():
    outputs = net(all_images)    
    loss_likelihood = loss_function(outputs, all_labels)
    reg=images_regulariser(net)
    loss=loss_likelihood+reg
    optimizer.zero_grad()
    loss.backward()
    return(loss)

  optimizer = torch.optim.LBFGS(net.parameters(), history_size=30, max_iter=20)
  for epoch in range(num_steps):
    optimizer.step(lpost)
  
  net_star=copy.deepcopy(net)
  len_params=len(list(net_star.parameters()))
  #Variance reduction - saving gradients at each batch at x_star
  net_star_grad_list=[]
  net_star_full_grad=[torch.zeros_like(par, device=device) for par in list(net_star.parameters())]
  for i in range(no_batches):
      images=images_list[i]
      labels=labels_list[i]
      outputs=net_star(images)
      loss_likelihood = loss_function(outputs, labels)
      #reg=images_regulariser(net)
      net_star.zero_grad()
      loss_likelihood.backward()
      grads=[par.grad*batch_size for par in list(net_star.parameters())]
      net_star_grad_list.append(grads)
      for g, gi in zip(net_star_full_grad,grads):
        g+=gi          

  len_params=len(list(net_star.parameters()))
  list_no_bias=torch.zeros(len_params)
  pit=0
  for name, p in net_star.named_parameters():
      if 'bias' not in name:
          list_no_bias[pit]=1.0
      pit+=1
  return net_star, net_star_grad_list, net_star_full_grad, len_params, list_no_bias

In [16]:
net_star, net_star_grad_list, net_star_full_grad, len_params, list_no_bias=find_MAP(40)

In [17]:
all_images=torch.cat(images_list,dim=0).detach()
all_labels=torch.cat(labels_list,dim=0).detach()

In [18]:
def SMS_HMC(num_steps,num_symmetric_sweeps,h,partial):
  net=copy.deepcopy(net_star)

  with torch.no_grad():
    test_labels_arr=torch.zeros(test_size).detach()
    test_prob_arr=torch.zeros([test_size,num_classes,num_steps]).detach()
  #Initialise velocities
    rejects=0  
    for par in net.parameters():
      par.v = torch.randn_like(par.data,device=device).detach()
    net_old=copy.deepcopy(net)
    for par_old,par in zip(net_old.parameters(),net.parameters()):
      par_old.v = par.v

    #ham_old=hamiltonian(net).detach()

    
  for step in range(num_steps):
    print("Step:", step, "Rejects:", rejects)
    rperm=random.permutation(list(range(no_batches))) 
    #h_factor=torch.rand(1).detach().cuda()      
    ham_old=hamiltonian(net).detach()    
    for s in range(num_symmetric_sweeps):
      for i in range(no_batches*2):
        ind=ind_create(i)
        leapfrogvxv_step(net,h,rperm[ind])
    r=torch.rand(1).detach().cuda()
    ham_new=hamiltonian(net)
    print("Hamiltonian diff:", ham_old-ham_new)    
    if(r.log()>(ham_old-ham_new)):
      net=copy.deepcopy(net_old)
      for par_old,par in zip(net_old.parameters(),net.parameters()):
        par.v = par_old.v
      rejects+=1
      for par in net.parameters():
        par.v = -par.v

    for par in net.parameters():
      par.v = par.v*partial+torch.randn_like(par.data,device=device).detach()*((1-partial**2)**(1/2))

    net_old=copy.deepcopy(net)
    for par_old,par in zip(net_old.parameters(),net.parameters()):
      par_old.v = par.v 
    
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,step]=torch.softmax(outputt,dim=1)    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Step [%d], Average Loss: %0.4f' %(step+1, ham_old/train_data_len))
  return(test_labels_arr,test_prob_arr,rejects)


def SMS_HMC2(num_steps,num_symmetric_sweeps,h,partial):
  net=copy.deepcopy(net_star)

  with torch.no_grad():
    test_labels_arr=torch.zeros(test_size).detach()
    test_prob_arr=torch.zeros([test_size,num_classes,num_steps]).detach()
  #Initialise velocities
    rejects=0  
    for par in net.parameters():
      par.v = torch.randn_like(par.data,device=device).detach()
    net_old=copy.deepcopy(net)
    for par_old,par in zip(net_old.parameters(),net.parameters()):
      par_old.v = par.v

    #ham_old=hamiltonian(net).detach()

    
  for step in range(num_steps):
    print("Step:", step, "Rejects:", rejects)
    rperm=random.permutation(list(range(no_batches))) 
    #h_factor=torch.rand(1).detach().cuda()      
    ham_old=hamiltonian(net).detach()    
    for s in range(num_symmetric_sweeps):
      for i in range(no_batches*2):
        ind=ind_create(i)
        if(i<(no_batches-1)):
          leapforward(net,h,rperm[ind])
        if(i>(no_batches)):
          leapbackward(net,h,rperm[ind])
        if(i==(no_batches-1) or i==(no_batches)):
          leaponlyv(net,h,rperm[ind])
    r=torch.rand(1).detach().cuda()
    ham_new=hamiltonian(net)
    print("Hamiltonian diff:", ham_old-ham_new)    
    if(r.log()>(ham_old-ham_new)):
      net=copy.deepcopy(net_old)
      for par_old,par in zip(net_old.parameters(),net.parameters()):
        par.v = par_old.v
      rejects+=1
      for par in net.parameters():
        par.v = -par.v
    # else:
    #   ham_old=ham_new
    for par in net.parameters():
      par.v = par.v*partial+torch.randn_like(par.data,device=device).detach()*((1-partial**2)**(1/2))

    net_old=copy.deepcopy(net)
    for par_old,par in zip(net_old.parameters(),net.parameters()):
      par_old.v = par.v 
    # ham_diff=0
    # for par in net.parameters():
    #   ham_diff-=par.v.pow(2).sum()/2
    #   par.v = torch.randn_like(par.data,device=device).detach()
    #   ham_diff+=par.v.pow(2).sum()/2
    # ham_old+=ham_diff
    
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        _, predictedt = torch.max(outputt,1)
        #_, predictedt2 = torch.max(outputt2,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,step]=torch.softmax(outputt,dim=1)    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Step [%d], Average Loss: %0.4f' %(step+1, ham_old/train_data_len))
  return(test_labels_arr,test_prob_arr,rejects)


def HMC(num_steps,num_leapfrog,h,partial):
  net=copy.deepcopy(net_star)

  with torch.no_grad():
    test_labels_arr=torch.zeros(test_size).detach()
    test_prob_arr=torch.zeros([test_size,num_classes,num_steps]).detach()
  #Initialise velocities
    rejects=0  
    for par in net.parameters():
      par.v = torch.randn_like(par.data,device=device).detach()

    #ham_old=hamiltonian(net).detach()

    
  for step in range(num_steps):
    print("Step:", step, "Rejects:", rejects)
    #h_factor=torch.rand(1).detach().cuda()      

    net_old=copy.deepcopy(net)
    for par_old,par in zip(net_old.parameters(),net.parameters()):
      par_old.v = par.v
    
    ham_old=hamiltonian(net_old).detach()    
    for s in range(num_leapfrog):
      leapfrog_step(net,h)
    r=torch.rand(1).detach().cuda()
    ham_new=hamiltonian(net).detach()
    print("Hamiltonian diff:", ham_old-ham_new)    
    if(r.log()>(ham_old-ham_new)):
      net=copy.deepcopy(net_old)
      for par_old,par in zip(net_old.parameters(),net.parameters()):
        par.v = par_old.v
      rejects+=1
      for par in net.parameters():
        par.v = -par.v


    for par in net.parameters():
      par.v = par.v*partial+torch.randn_like(par.data,device=device).detach()*((1-partial**2)**(1/2))
        
    correct = 0
    total = 0  

    with torch.no_grad():
      for testit in range(test_no_batches):
        imagest=test_images_list[testit]
        labelst=test_labels_list[testit]
        actual_batch_size=len(imagest)
        test_labels_arr[(testit*batch_size):(testit*batch_size+actual_batch_size)]=labelst.detach().cpu()
        outputt = net(imagest).detach()#.reshape(actual_batch_size).detach()
        _, predictedt = torch.max(outputt,1)
        correct += (predictedt == labelst).sum()
        total += labelst.size(0)

        test_prob_arr[(testit*batch_size):(testit*batch_size+actual_batch_size),0:num_classes,step]=torch.softmax(outputt,dim=1)    
    #net.train()       
    print('Test accuracy of the model: %.3f %%' %((100*correct)/(total+1)))
    print('Step [%d], Average Loss: %0.4f' %(step+1, ham_old/train_data_len))
  return(test_labels_arr,test_prob_arr,rejects)

In [19]:
h=torch.tensor(2e-3)
num_steps=int(100)
num_leapfrog=100


test_labels_arr,test_prob_arr,rejects=HMC(num_steps,num_leapfrog,h,0.7)


Step: 0 Rejects: 0
Hamiltonian diff: tensor(-0.9199, device='cuda:0')
Test accuracy of the model: 84.172 %
Step [1], Average Loss: 0.4549
Step: 1 Rejects: 0
Hamiltonian diff: tensor(-0.0254, device='cuda:0')
Test accuracy of the model: 84.172 %
Step [2], Average Loss: 0.4722
Step: 2 Rejects: 1
Hamiltonian diff: tensor(-1.0879, device='cuda:0')
Test accuracy of the model: 84.172 %
Step [3], Average Loss: 0.4687
Step: 3 Rejects: 2
Hamiltonian diff: tensor(-0.4160, device='cuda:0')
Test accuracy of the model: 84.172 %
Step [4], Average Loss: 0.4771
Step: 4 Rejects: 3
Hamiltonian diff: tensor(-0.4609, device='cuda:0')
Test accuracy of the model: 84.052 %
Step [5], Average Loss: 0.4751
Step: 5 Rejects: 3
Hamiltonian diff: tensor(-1.1191, device='cuda:0')
Test accuracy of the model: 84.052 %
Step [6], Average Loss: 0.4897
Step: 6 Rejects: 4
Hamiltonian diff: tensor(0.0059, device='cuda:0')
Test accuracy of the model: 83.802 %
Step [7], Average Loss: 0.4938
Step: 7 Rejects: 4
Hamiltonian diff

KeyboardInterrupt: 

In [20]:
h=torch.tensor(6e-5)
num_steps=int(100)
num_symmetric_sweeps=10

test_labels_arr,test_prob_arr,rejects=SMS_HMC(num_steps,num_symmetric_sweeps,h,0.7)


Step: 0 Rejects: 0
Hamiltonian diff: tensor(-1.1172, device='cuda:0')
Test accuracy of the model: 84.222 %
Step [1], Average Loss: 0.4563
Step: 1 Rejects: 0
Hamiltonian diff: tensor(0.5664, device='cuda:0')
Test accuracy of the model: 84.072 %
Step [2], Average Loss: 0.4605
Step: 2 Rejects: 0
Hamiltonian diff: tensor(-0.2871, device='cuda:0')
Test accuracy of the model: 84.072 %
Step [3], Average Loss: 0.4690
Step: 3 Rejects: 1
Hamiltonian diff: tensor(0.2324, device='cuda:0')
Test accuracy of the model: 84.172 %
Step [4], Average Loss: 0.4684
Step: 4 Rejects: 1
Hamiltonian diff: tensor(-1.1016, device='cuda:0')
Test accuracy of the model: 84.172 %
Step [5], Average Loss: 0.4727
Step: 5 Rejects: 2
Hamiltonian diff: tensor(0.2422, device='cuda:0')
Test accuracy of the model: 84.192 %
Step [6], Average Loss: 0.4738
Step: 6 Rejects: 2
Hamiltonian diff: tensor(-1.1816, device='cuda:0')
Test accuracy of the model: 84.192 %
Step [7], Average Loss: 0.4788
Step: 7 Rejects: 3
Hamiltonian diff: 

KeyboardInterrupt: 

In [ ]:
2e-3/6e-5

33.333333333333336

In [ ]:
test_prob=torch.Tensor(test_prob_arr[:,:,(1*num_steps//5):num_steps]).mean(-1).reshape(test_size,num_classes)
rps_arr=(rps_calc(test_prob, test_labels_arr).mean())
nll_arr=nll_calc(test_prob, test_labels_arr)
ace_arr=adaptive_calibration_error(test_prob, test_labels_arr).mean()
_, predictedt = torch.max(test_prob,1)
acc_arr=(predictedt==test_labels_arr.reshape(1,test_size)).sum()/test_size
